In [28]:
import findspark
findspark.init()

import pyspark

from pyspark.sql.functions import expr,col,mean,variance
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RFormula
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

In [5]:
csv=(r"C:\Users\chanc\Downloads\zoo.csv")
   
from pyspark import SparkFiles
spark.sparkContext.addFile(csv)
zoo=spark.read.csv(SparkFiles.get("zoo.csv"), header=True, inferSchema= True)
zoo.show(5)

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|
|      bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|
|      boar|   1|       0|   0|   1|       0|      0|  

In [6]:
zoo=zoo.withColumn("ismammal",expr("CASE WHEN TYPE =1 THEN 1 ELSE 0 END"))
zoo.show(5)

+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+
|AnimalName|Hair|Feathers|Eggs|Milk|Airborne|Aquatic|Predator|Toothed|Backbone|Breathes|Venomous|Fins|Legs|Tail|Domestic|Catsize|Type|ismammal|
+----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----+--------+
|  aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|       1|
|  antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|   1|       1|
|      bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|   4|       0|
|      bear|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|   1|  

In [11]:
preprocessed_data= RFormula(formula="ismammal ~ Hair + Feathers + Eggs + Milk + Airborne + Aquatic + Predator + Toothed + Backbone + Breathes + Venomous + Fins + Legs + Tail + Domestic + Catsize")
preprocessed_data=preprocessed_data.fit(zoo)
preprocessed_data=preprocessed_data.transform(zoo)
preprocessed_data.select("features","label").show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(16,[0,3,6,7,8,9,...|  1.0|
|(16,[0,3,7,8,9,12...|  1.0|
|(16,[2,5,6,7,8,11...|  0.0|
|(16,[0,3,6,7,8,9,...|  1.0|
|(16,[0,3,6,7,8,9,...|  1.0|
|(16,[0,3,7,8,9,12...|  1.0|
|(16,[0,3,7,8,9,12...|  1.0|
|(16,[2,5,7,8,11,1...|  0.0|
|(16,[2,5,6,7,8,11...|  0.0|
|(16,[0,3,7,8,9,12...|  1.0|
+--------------------+-----+
only showing top 10 rows



# splitting data into 70 % training and 30 %

In [12]:
train, test=preprocessed_data.randomSplit([0.7,0.3])

In [13]:
lr=LogisticRegression(labelCol="label", featuresCol="features")

In [14]:
trainedLR=lr.fit(train)

In [18]:
result=trainedLR.transform(train)
result.select("AnimalName","label","prediction").show()

+----------+-----+----------+
|AnimalName|label|prediction|
+----------+-----+----------+
|      bass|  0.0|       0.0|
|      boar|  1.0|       1.0|
|   buffalo|  1.0|       1.0|
|      carp|  0.0|       0.0|
|      cavy|  1.0|       1.0|
|   cheetah|  1.0|       1.0|
|      chub|  0.0|       0.0|
|      clam|  0.0|       0.0|
|  crayfish|  0.0|       0.0|
|      crow|  0.0|       0.0|
|      deer|  1.0|       1.0|
|   dogfish|  0.0|       0.0|
|   dolphin|  1.0|       1.0|
|      dove|  0.0|       0.0|
|      duck|  0.0|       0.0|
|  elephant|  1.0|       1.0|
|      flea|  0.0|       0.0|
|  fruitbat|  1.0|       1.0|
|   giraffe|  1.0|       1.0|
|      girl|  1.0|       1.0|
+----------+-----+----------+
only showing top 20 rows

